In [1]:
import numpy as np
from numpy import genfromtxt

In [2]:
data = genfromtxt('train.csv',delimiter=',')
label = genfromtxt('label.csv',delimiter=',')
test = genfromtxt('test.csv',delimiter=',')
test[0][0] = 1

In [3]:
test

array([[ 1.0000e+00,  9.9539e-01, -5.8890e-02,  8.5243e-01,  2.3060e-02,
         8.3398e-01, -3.7708e-01,  1.0000e+00,  3.7600e-02,  8.5243e-01,
        -1.7755e-01,  5.9755e-01, -4.4945e-01,  6.0536e-01, -3.8223e-01,
         8.4356e-01, -3.8542e-01,  5.8212e-01, -3.2192e-01,  5.6971e-01,
        -2.9674e-01,  3.6946e-01, -4.7357e-01,  5.6811e-01, -5.1171e-01,
         4.1078e-01, -4.6168e-01,  2.1266e-01, -3.4090e-01,  4.2267e-01,
        -5.4487e-01,  1.8641e-01, -4.5300e-01],
       [ 1.0000e+00,  1.0000e+00, -1.8829e-01,  9.3035e-01, -3.6156e-01,
        -1.0868e-01, -9.3597e-01,  1.0000e+00, -4.5490e-02,  5.0874e-01,
        -6.7743e-01,  3.4432e-01, -6.9707e-01, -5.1685e-01, -9.7515e-01,
         5.4990e-02, -6.2237e-01,  3.3109e-01, -1.0000e+00, -1.3151e-01,
        -4.5300e-01, -1.8056e-01, -3.5734e-01, -2.0332e-01, -2.6569e-01,
        -2.0468e-01, -1.8401e-01, -1.9040e-01, -1.1593e-01, -1.6626e-01,
        -6.2880e-02, -1.3738e-01, -2.4470e-02],
       [ 1.0000e+00,  1.0000

In [4]:
label_neg = np.where(label == -1)
label[label_neg] = 0

In [5]:
label

array([0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
       0., 1., 0., 1., 0.

In [6]:
def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
  
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]

    converged = False
    
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]

In [7]:
class RBF:

    def __init__(self, X, y, tX, ty, pred_in, num_of_classes, k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.pred_in = pred_in

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def rbf(self, x, c, s):
        distance = get_distance(x, c)
        return np.exp(-distance / s ** 2)

    def rbf_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)

    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, max_iters=1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.rbf_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.rbf_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty
        print(diff)

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

        return self.pred_ty, self.w

    def predit(self):
        test_pred, self.weight = self.fit()
        RBF_list_pred = self.rbf_list(self.pred_in, self.centroids, self.std_list)
        self.pred_output = RBF_list_pred @ self.weight
        self.pred_output = np.array([np.argmax(x) for x in self.pred_output])

        return self.pred_output       


In [48]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.1, random_state=2100886)

In [49]:
print('Length of Testing data is ',len(test_x))
print('Length of Training data is ',len(train_x))

Length of Testing data is  33
Length of Training data is  297


In [50]:
RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, test, num_of_classes=2,
                     k=10, std_from_clusters=False)

pred = RBF_CLASSIFIER.predit()

K-MEANS:  6
K-MEANS:  3
K-MEANS:  0
K-MEANS:  1
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
Accuracy:  0.9393939393939394


In [16]:
for i in range (1,26):
    print(f'Accuracy for k = {i}')
    RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, test, num_of_classes=2,
                     k=i, std_from_clusters=False)
    RBF_CLASSIFIER.fit()
    print('\n')

Accuracy for k = 1
K-MEANS:  4
K-MEANS:  0
[ 0.  0.  0. -1.  0.  0. -1.  0. -1. -1. -1.  0.  0. -1. -1. -1. -1.  0.
 -1.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1.  0. -1.  0. -1.  0. -1. -1.
 -1.  0.  0. -1. -1. -1.  0. -1. -1. -1. -1.  0.  0. -1.  0. -1. -1. -1.
  0. -1. -1. -1. -1. -1.  0.  0. -1. -1. -1. -1.]
Accuracy:  0.3333333333333333


Accuracy for k = 2
K-MEANS:  5
K-MEANS:  2
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 1.  1.  1. -1.  1.  0.  0.  1.  0.  0.  0.  1.  1. -1.  0.  0.  0.  1.
  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  1.  0.  0.
  0.  1.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  1.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
Accuracy:  0.6363636363636364


Accuracy for k = 3
K-MEANS:  7


s:\anaconda\envs\neural_fuzzy\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


K-MEANS:  3
K-MEANS:  1
K-MEANS:  1
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  1. -1.  0.  0.  1. -1.  0.
  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  1.  0.  1.  0.  0.  0.
  1.  0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.]
Accuracy:  0.7878787878787878


Accuracy for k = 4
K-MEANS:  3
K-MEANS:  3
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  1.  0.  0.
  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.
  1.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0. -1.]
Accuracy:  0.7727272727272727


Accuracy for k = 5
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  1.  0. -1.  0.  0

K-MEANS:  0
K-MEANS:  5
K-MEANS:  3
K-MEANS:  2
K-MEANS:  1
K-MEANS:  0
[ 1.  0.  0. -1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  1.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
Accuracy:  0.8939393939393939


Accuracy for k = 22
K-MEANS:  12
K-MEANS:  1
K-MEANS:  3
K-MEANS:  2
K-MEANS:  0
K-MEANS:  1
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
K-MEANS:  0
[ 0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
Accuracy:  0.9242424242424242


Accuracy for k = 23
K-MEANS:  0
K-MEANS:  1
K-MEANS:  0
K-MEANS:  1
K-MEANS:  2
K-MEANS:  0
K-MEANS:  0
K-MEANS: 

In [52]:
print(len(pred))
pred

21


array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1],
      dtype=int64)

In [47]:
pred_neg = np.where(pred == 0)
pred[pred_neg] = -1
pred #Converted back to originl class label of 1,-1

array([ 1, -1,  1, -1,  1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1,  1,
       -1,  1, -1,  1], dtype=int64)

In [50]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(train_x, train_y)

y_pred = svclassifier.predict(test_x)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(test_y ,y_pred))
print(classification_report(test_y ,y_pred))

[[ 4  0]
 [ 0 13]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         4
         1.0       1.00      1.00      1.00        13

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17



In [51]:
y_svm = svclassifier.predict(test)

In [67]:
y_svm

21

In [53]:
y_svm_negative = np.where(y_svm == 0 )
y_svm[y_svm_negative] = -1
y_svm

array([ 1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,
        1.,  1., -1.,  1., -1.,  1., -1.,  1.])